In [10]:
import sqlite3
import requests
import time

import os

In [11]:

WB_TOKEN = os.getenv('WB_TOKEN')
DB_NAME = "zip.db"

In [12]:
DB_NAME

'zip.db'

In [13]:
# Параметры подключения к базе

WB_API_URL = "https://discounts-prices-api.wildberries.ru/api/v2/upload/task"
HEADERS = {"Authorization": f"Bearer {WB_TOKEN}", "Content-Type": "application/json"}

# Подключение к базе и выборка записей
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

query = """
    SELECT nm_id, new_price, new_discount
    FROM products
    WHERE available_count = 5
"""
cursor.execute(query)
rows = cursor.fetchall()
conn.close()

# Формируем список товаров для обновления цен и скидок
goods_data = [{"nmID": nm_id, "price": new_price, "discount": new_discount} for nm_id, new_price, new_discount in rows]

# Отправка данных батчами по 1000 товаров (API поддерживает максимум 1000 за запрос)


In [14]:
BATCH_SIZE = 1000
REQUEST_LIMIT = 10  # Максимум 10 запросов за 6 секунд
REQUEST_INTERVAL = 6 / REQUEST_LIMIT  # Интервал между запросами

for i in range(0, len(goods_data), BATCH_SIZE):
    batch = goods_data[i:i + BATCH_SIZE]

    payload = {"data": batch}
    response = requests.post(WB_API_URL, json=payload, headers=HEADERS)

    if response.status_code == 200:
        print(f"✅ Успешно обновлены цены и скидки для {len(batch)} товаров")
    else:
        print(f"❌ Ошибка при обновлении: {response.text}")

    time.sleep(REQUEST_INTERVAL)  # Задержка для соблюдения лимита API


✅ Успешно обновлены цены и скидки для 8 товаров


In [15]:

WB_API_N_URL = "https://marketplace-api.wildberries.ru/api/v3/stocks/{warehouseId}"
WAREHOUSE_ID = 1283008


In [18]:
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

query = """
    SELECT skus, pcs, available_count
    FROM products
    WHERE skus IS NOT NULL
"""
cursor.execute(query)
rows = cursor.fetchall()
conn.close()

# Формируем список остатков
stocks_data = [
    {
        "sku": skus,
        "amount": (
            10 if available_count == 5 else
            0
        )
    }
    for skus, pcs, available_count in rows
]

In [19]:
stocks_data

[{'sku': '2042726157567', 'amount': 10},
 {'sku': '2042726157376', 'amount': 10},
 {'sku': '2042726157543', 'amount': 10},
 {'sku': '2042726157444', 'amount': 10},
 {'sku': '2042726157451', 'amount': 10},
 {'sku': '2042726157468', 'amount': 10},
 {'sku': '2042726157536', 'amount': 10},
 {'sku': '2042726157390', 'amount': 10},
 {'sku': '2042726157406', 'amount': 0},
 {'sku': '2042726157482', 'amount': 0},
 {'sku': '2042726157505', 'amount': 0},
 {'sku': '2042726157499', 'amount': 0},
 {'sku': '2042726157369', 'amount': 0},
 {'sku': '2042726157437', 'amount': 0},
 {'sku': '2042726157420', 'amount': 0},
 {'sku': '2042726157529', 'amount': 0},
 {'sku': '2042726157475', 'amount': 0},
 {'sku': '2042726157512', 'amount': 0},
 {'sku': '2042726157550', 'amount': 0}]

In [20]:
BATCH_SIZE = 1000
REQUEST_LIMIT = 300  # Максимум 300 запросов в минуту
REQUEST_INTERVAL = 60 / REQUEST_LIMIT  # Интервал между запросами

for i in range(0, len(stocks_data), BATCH_SIZE):
    batch = stocks_data[i:i + BATCH_SIZE]

    payload = {"stocks": batch}
    url = WB_API_N_URL.format(warehouseId=WAREHOUSE_ID)
    response = requests.put(url, json=payload, headers=HEADERS)

    if response.status_code == 204:
        print(f"✅ Успешно обновлены остатки для {len(batch)} товаров")
    else:
        print(f"❌ Ошибка при обновлении: {response.text}")

    time.sleep(REQUEST_INTERVAL)  # Задержка для соблюдения лимита API

✅ Успешно обновлены остатки для 19 товаров
